# N-Gram Language Models
In this exercise, we will use n-gram language models to predict the probability of text, and generate it.

In [29]:
from multiprocessing.managers import Value

import nltk
from nltk.corpus import gutenberg

First, we load Jane Austen's Emma from NLTK's gutenberg corpus that we also used in a previous exercise. Tokenize and lowercase this text such that we have a list of words.

In [30]:
raw_text = gutenberg.raw('austen-emma.txt')
words = [w.lower() for w in nltk.word_tokenize(raw_text)]
len(words)

191855

Write an n-gram language model class that takes the word list and a parameter `n` as inputs, where `n` is a positive integer larger than 1 that determines the `n` of the n-gram LM. The LM should build a dictionary of n-gram counts from the word list.

In [31]:
def preprocess(string: str) -> list[str]:
    return [w.lower() for w in nltk.word_tokenize(string)]

In [32]:
from collections import defaultdict

class NGramLanguageModel:
    
    def __init__(self, words, n):
        assert n > 1, "n needs to be a positive integer > 1"
        assert n <= len(words), "n can't be larger than the number of words"
        self.counts: dict[tuple[str, ...], int] = defaultdict(int)
        self.n = n
        for i in range(len(words) - n + 1):
            ngram = tuple(words[i : i+n])
            self.counts[ngram] += 1
            self.counts[ngram[:-1]] += 1
        self.counts[ngram[1:]] += 1

Now we "train" the n-gram LM by building the n-gram counts of the Emma novel. Use a low `n` (i.e. 2 or 3).

In [56]:
lm = NGramLanguageModel(words, 3)

Let's add a method `log_probability` to the n-gram LM class that computes the probability of an input string. Since multiplying many probabilities (<= 1) results in very small numbers that can underflow, we sum the log probabilities instead.

In [57]:
import math

def log_probability(self, input_string) -> float:
        """ Returns the log-probability of the input string."""
        input_words = preprocess(input_string)
        probability = []
        for i in range(len(input_words) - self.n + 1):
            ngram = tuple(input_words[i : i + self.n])
            if ngram in self.counts:
                probability.append(math.log(self.counts[ngram] / self.counts[ngram[:-1]]))
        return sum(probability) / len(probability)

NGramLanguageModel.log_probability = log_probability

Shorter texts will have higher log probability than longer texts, so we need to normalize it by the number of words in the input string.

In [58]:
print(lm.log_probability("What is the meaning of life?"))
print(lm.log_probability("What is the meaning of life, given I am a student?"))

-2.208105236360199
-2.420144330727304


Lets predict the probabilities of two novels under our trained model: Jane Austen's *Sense and Sensibility* (`austen-sense.txt`) and Shakespeare's *Hamlet* (`shakespeare-hamlet.txt`).
- What do you expect will happen?
- What do you observe?

In [59]:
austen_sense = gutenberg.raw('austen-sense.txt')
shakespeare_hamlet = gutenberg.raw('shakespeare-hamlet.txt')

print(f"Austen Emma: {lm.log_probability(raw_text)}")
print("Austen Sense: ", lm.log_probability(austen_sense))
print("Shakespeare Hamlet: ", lm.log_probability(shakespeare_hamlet))

Austen Emma: -1.7492105638273363
Austen Sense:  -2.405643228181783
Shakespeare Hamlet:  -2.7892762868035343


How many n-grams are known in each input?

In [60]:
def count_known_n_grams(self, input_string) -> tuple[int, int]:
    input_words = preprocess(input_string)
    count = 0
    for i in range(len(input_words) - self.n + 1):
        ngram = tuple(input_words[i : i + self.n])
        if ngram in self.counts:
            count += 1
    return count, len(input_words) - self.n + 1

NGramLanguageModel.count_known_n_grams = count_known_n_grams

In [61]:
known, total = lm.count_known_n_grams(raw_text)
print(f"Austen Emma: {known/total:.2%}, {known}/{total}")
known, total = lm.count_known_n_grams(austen_sense)
print(f"Austen Sense: {known/total:.2%}, {known}/{total}")
known, total = lm.count_known_n_grams(shakespeare_hamlet)
print(f"Shakespeare Hamlet: {known/total:.2%}, {known}/{total}")

Austen Emma: 100.00%, 191853/191853
Austen Sense: 29.56%, 41806/141438
Shakespeare Hamlet: 8.82%, 3211/36409


Let's add a method `generate` that takes the start of a sentence ("prompt") and a number of words to generate, then continues our prompt.

In [63]:
import random

def generate(self, prompt, num_words=10):
    """ Continues a text starting with `prompt` for the `num_words` next words. """
    words = preprocess(prompt)
    for _ in range(num_words):
        prefix = tuple(words[-self.n + 1:])
        if prefix not in self.counts:
            raise ValueError(f"Unknown n-gram (-1): {prefix}")
        next_word_dist = {}
        for n_gram in self.counts:
            if len(n_gram) == self.n and n_gram[:-1] == prefix:
                next_word_dist[n_gram] = self.counts[n_gram] / self.counts[prefix]
        #print(prefix, next_word_dist)
        best_ngram, prob = max(next_word_dist.items(), key=lambda x: x[1])
        print(best_ngram, prob)
        words.append(best_ngram[-1])
    return " ".join(words)
        

NGramLanguageModel.generate = generate

Play around with a few different prompts.

In [67]:
output = lm.generate("i went for a walk", 50)
print(output)

('a', 'walk', ',') 0.25
('walk', ',', 'or') 0.1111111111111111
(',', 'or', 'any') 0.07103825136612021
('or', 'any', 'thing') 0.35294117647058826
('any', 'thing', 'to') 0.08536585365853659
('thing', 'to', 'be') 0.25925925925925924
('to', 'be', 'sure') 0.04132231404958678
('be', 'sure', ',') 0.23076923076923078
('sure', ',', "''") 0.18181818181818182
(',', "''", 'said') 0.3852739726027397
("''", 'said', 'he') 0.2188679245283019
('said', 'he', ',') 0.5797101449275363
('he', ',', '``') 0.35714285714285715
(',', '``', 'i') 0.152
('``', 'i', 'am') 0.16091954022988506
('i', 'am', 'sure') 0.2715736040609137
('am', 'sure', 'i') 0.16822429906542055
('sure', 'i', 'should') 0.19047619047619047
('i', 'should', 'not') 0.21100917431192662
('should', 'not', 'have') 0.35
('not', 'have', 'been') 0.14705882352941177
('have', 'been', 'a') 0.07883817427385892
('been', 'a', 'great') 0.08888888888888889
('a', 'great', 'deal') 0.4740740740740741
('great', 'deal', 'of') 0.40625
('deal', 'of', 'the') 0.10256410